In [139]:
import os
from os.path import join
from copy import deepcopy
import numpy as np
import pickle
import scipy
from sklearn.linear_model import LinearRegression

import torch
import sbi
from sbi import utils as utils
from sbi.inference import SNLE, likelihood_estimator_based_potential

import yaml
import argparse

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [153]:
get_cen = lambda edges: np.array([(edges[i]+edges[i+1])/2. for i in range(len(edges)-1)])
def gapper(v):
    """ Returns the gapper velocity dispersion of a cluster (Sigma_G)

    v is an array of galaxy velocity values.
    """
    n = len(v)
    w = np.arange(1, n) * np.arange(n-1, 0, -1)
    g = np.diff(v)
    sigG = (np.sqrt(np.pi))/(n*(n-1)) * np.dot(w, g)
    return sigG

# Get config

In [264]:
cfgpath = 'configs/FS2dC50.yaml'

In [265]:
with open(cfgpath, 'r') as f:
    cfg = dict(yaml.safe_load(f))
print(cfg)
param_names = cfg['param_names']
data_names = cfg['data_names']

{'data': {'name': 'FS2dC50'}, 'param_names': ['logm200', 'logr200'], 'data_names': ['x', 'y', 'vrf'], 'priors': {'low': [13.9, -0.31], 'high': [14.8, 0.05]}, 'val': {'sample_at': [0.5], 'Nclu': 100, 'Nsamp': 100}, 'example': {'sample_at': [0.5], 'Nclu': 5, 'Nsamp': 1000}}


# Load

In [266]:
datapath = join('data/processed', cfg['data']['name'])
print('Loading from:', datapath)
data = np.load(join(datapath, 'x.npy'))
theta = np.load(join(datapath, 'theta.npy'))
fold = np.load(join(datapath, 'fold.npy'))
ids = np.load(join(datapath, 'id.npy'))

Loading from: data/processed/FS2dC50


In [267]:
modelpath = join('saved_models', cfg['data']['name'])
print('Loading from:', modelpath)
samps = np.load(join(modelpath, 'samps.npy'))
maps = np.load(join(modelpath, 'MAP.npy'))
trues = np.load(join(modelpath, 'trues.npy'))

Loading from: saved_models/FS2dC50


In [268]:
# fit M-sigma linear regression
mask = fold != 0
dtr = data[mask,-1]
ttr = theta[mask,0]
itr = ids[mask]

sigvs = []
for i in np.unique(itr):
    tmp = gapper(dtr[itr==i])
    sigvs.append((tmp, ttr[itr==i][0]))
sigvs = np.array(sigvs)
sigvs = sigvs[sigvs[:,0]>0]

lr = LinearRegression()
lr = lr.fit(sigvs[:,1][:,None], np.log10(sigvs[:,0]))

In [269]:
# apply on test set
mask = fold == 0
dtr = data[mask,-1]
ttr = theta[mask,0]
itr = ids[mask]

sigvs = []
for i in np.unique(itr):
    tmp = gapper(dtr[itr==i])
    sigvs.append((tmp, ttr[itr==i][0]))
sigvs = np.array(sigvs)
sigvs = sigvs[sigvs[:,0]>0]

svtrue = sigvs[:,1]
svpred = (np.log10(sigvs[:,0])-lr.intercept_)/lr.coef_
# censor within prior
svpred = np.minimum(cfg['priors']['high'][0], svpred)
svpred = np.maximum(cfg['priors']['low'][0], svpred)

In [270]:
stdpr = np.sqrt((cfg['priors']['high'][0] - cfg['priors']['low'][0])**2/12)
stdsv = np.std(svtrue-svpred)
stdsamp = np.std(trues[:,None,:].repeat(repeats=samps.shape[1], axis=1)[...,0]-samps[...,0])
stdmean = np.std(trues[...,0]-samps[...,0].mean(axis=1))
stdmap = np.std(trues[...,0]-maps[:,0,0])

stdlist = [stdpr, stdsv, stdsamp, stdmean, stdmap]
stdlist = [f"{x:.3f}" for x in stdlist]
print(' & '.join(stdlist))

0.260 & 0.417 & 0.200 & 0.169 & 0.200
